- This code identified tables' bounding boxes and stores it in form of images.
- Took 1 min 30 sec for a 121 pager PDF.

In [1]:
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
from pdf2image import convert_from_path
import fitz
from PIL import Image
import os

c:\Users\sarve\Documents\Personal Experiments\Table Extraction\.table_extraction\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdf_path = "../Input/AAPL 2024 10k.pdf"
output_folder = "./Output/tables"
os.makedirs(output_folder, exist_ok=True)

In [3]:
model_path = hf_hub_download(repo_id="foduucom/table-detection-and-extraction", filename="best.pt")
model = YOLO(model_path, task='detect')

In [4]:
doc = fitz.open(pdf_path)
total_tables = 0

for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    pix = page.get_pixmap(dpi=300)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Save temporary image for inference
    temp_img_path = f"temp_page_{page_num+1}.png"
    img.save(temp_img_path)

    # Run inference
    results = model.predict(source=temp_img_path)

    # Crop and save each detected table
    table_count = 0
    for res in results:
        for bbox in res.boxes.xyxy:
            x1, y1, x2, y2 = map(int, bbox.tolist())
            table_img = img.crop((x1, y1, x2, y2))
            table_count += 1
            total_tables += 1
            table_path = os.path.join(output_folder, f"page{page_num+1}_table{table_count}.png")
            table_img.save(table_path)
            print(f"Saved: {table_path}")

    # Remove temporary image
    os.remove(temp_img_path)

if total_tables == 0:
    print("No tables detected in the PDF.")
else:
    print(f"✅ {total_tables} table images saved in {output_folder}")


image 1/1 c:\Users\sarve\Documents\Personal Experiments\Table Extraction\yolov8\temp_page_1.png: 640x512 1 borderless, 306.5ms
Speed: 5.3ms preprocess, 306.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 512)
Saved: ./Output/tables\page1_table1.png

image 1/1 c:\Users\sarve\Documents\Personal Experiments\Table Extraction\yolov8\temp_page_2.png: 640x512 2 bordereds, 241.8ms
Speed: 3.9ms preprocess, 241.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 512)
Saved: ./Output/tables\page2_table1.png
Saved: ./Output/tables\page2_table2.png

image 1/1 c:\Users\sarve\Documents\Personal Experiments\Table Extraction\yolov8\temp_page_3.png: 640x512 1 borderless, 409.7ms
Speed: 5.9ms preprocess, 409.7ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 512)
Saved: ./Output/tables\page3_table1.png

image 1/1 c:\Users\sarve\Documents\Personal Experiments\Table Extraction\yolov8\temp_page_4.png: 640x512 (no detections), 291.8ms
Speed: 5.4ms preprocess, 291.8ms